In [1]:
# Run in a notebook cell with a leading ! or in terminal
!pip install pandas numpy ipywidgets python-dotenv openai
# enable widgets extension (if you're using classic Jupyter)
# For JupyterLab you may need labextension; but many JupyterLab installs already support ipywidgets.



   ---------- ----------------------------- 1/4 [uri-template]
   -------------------- ------------------- 2/4 [fqdn]
   ------------------------------ --------- 3/4 [isoduration]
   ---------------------------------------- 4/4 [isoduration]



In [3]:
import os
import pandas as pd
import numpy as np
from datetime import datetime
from textwrap import dedent
from IPython.display import display, Markdown, clear_output

# For interactive widgets (preferred UI)
try:
    import ipywidgets as widgets
    WIDGETS_AVAILABLE = True
except Exception:
    WIDGETS_AVAILABLE = False

# Optional OpenAI for polishing text (only if you set OPENAI_API_KEY)
from dotenv import load_dotenv
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
openai = None
if OPENAI_API_KEY:
    try:
        import openai as _openai
        _openai.api_key = OPENAI_API_KEY
        openai = _openai
    except Exception:
        openai = None

print("Widgets available:", WIDGETS_AVAILABLE)
print("OpenAI configured:", bool(openai))


Widgets available: True
OpenAI configured: False


In [5]:
def score_policy_for_user(policy_row, user_profile):
    score = 0.0
    reasons = []
    age = user_profile['age']
    goal = user_profile['goal'].lower()
    risk_tol = user_profile['risk_tolerance'].lower()
    income = user_profile['income']
    dependents = user_profile['dependents']
    existing_cover = user_profile['existing_cover']
    horizon = user_profile['time_horizon_years']
    
    # Age fit
    if age < policy_row['min_age'] or age > policy_row['max_age']:
        reasons.append("age_out_of_range")
        score -= 5.0
    else:
        score += 1.0
    
    # Goal fit (substring matching)
    if any(goal in g.lower() for g in policy_row['goal_fit']):
        reasons.append("goal_match")
        score += 4.0
    else:
        score += 0.5
    
    # Risk alignment
    map_risk = {"low":0, "medium":1, "high":2}
    policy_risk_val = {"low":0, "medium":1, "high":2}.get(policy_row['risk'],1)
    user_risk_val = map_risk.get(risk_tol,0)
    if policy_risk_val <= user_risk_val:
        score += 2.0
        reasons.append("risk_ok")
    else:
        score -= 1.0
        reasons.append("risk_high_for_user")
    
    # ULIP needs horizon
    if policy_row['policy_type'].lower() == "ulip":
        if horizon >= 7:
            score += 2.0
            reasons.append("horizon_ok")
        else:
            score -= 1.0
            reasons.append("horizon_short")
    
    # Term if protection needed
    if policy_row['policy_type'].lower() == "term":
        needed_cover = user_profile['income'] * 10  # simple proxy
        if existing_cover < needed_cover and dependents > 0:
            score += 3.0
            reasons.append("term_recommended_for_protection")
        else:
            score += 0.5
    
    # Income affordability small effect
    if income >= 500000:
        score += 0.5
    elif income < 200000:
        score -= 0.5
    
    # Extra boost for exact goal match
    if any(goal in g.lower() for g in policy_row['goal_fit']):
        score += 1.0
    
    return float(score), reasons

def recommend_policies(user_profile, catalog_df, top_k=3):
    results = []
    for _, row in catalog_df.iterrows():
        s, reasons = score_policy_for_user(row, user_profile)
        results.append({
            "policy_name": row['policy_name'],
            "policy_type": row['policy_type'],
            "score": s,
            "reasons": reasons,
            "description": row['description']
        })
    res_df = pd.DataFrame(results).sort_values("score", ascending=False).reset_index(drop=True)
    # confidence scale 0-100 based on min/max
    min_s, max_s = res_df['score'].min(), res_df['score'].max()
    if max_s - min_s < 1e-8:
        res_df['confidence_pct'] = 100.0
    else:
        res_df['confidence_pct'] = ((res_df['score'] - min_s) / (max_s - min_s)) * 100
    return res_df.head(top_k)


In [7]:
def format_explanation(user_profile, rec_row):
    name = user_profile['name'].split()[0]
    policy = rec_row['policy_name']
    ptype = rec_row['policy_type']
    desc = rec_row['description']
    confidence = round(rec_row['confidence_pct'], 1)
    reasons = ", ".join(rec_row['reasons'])
    
    plain = dedent(f"""
    Dear {name},

    Recommended: {policy} ({ptype}) — Confidence: {confidence}%

    Why this fits:
    - {desc}
    - Key reasons: {reasons}

    Suggestion: Consider this if your primary goal is '{user_profile['goal']}' and your risk preference is '{user_profile['risk_tolerance']}'.
    """).strip()
    
    # If OpenAI is configured, ask model to make it concise & friendly
    if openai:
        try:
            prompt = dedent(f"""
            Rewrite the following insurance recommendation into a friendly, concise customer note (max 70 words). Keep the facts and tone professional.

            Recommendation:
            {plain}
            """)
            resp = openai.ChatCompletion.create(
                model="gpt-4o-mini",
                messages=[{"role":"system","content":"You are a helpful insurance advisor."},
                          {"role":"user","content":prompt}],
                max_tokens=120,
                temperature=0.2
            )
            polished = resp.choices[0].message.content.strip()
            return polished
        except Exception:
            return plain
    else:
        return plain


In [9]:
def save_recommendation(user_profile, rec_df, filename="recommendations_log.csv"):
    top = rec_df.iloc[0].to_dict()
    row = {
        "timestamp": datetime.utcnow().isoformat(),
        "user_name": user_profile['name'],
        "age": user_profile['age'],
        "goal": user_profile['goal'],
        "risk_tolerance": user_profile['risk_tolerance'],
        "recommended_policy": top['policy_name'],
        "policy_type": top['policy_type'],
        "score": top['score'],
        "confidence_pct": top['confidence_pct'],
        "reasons": ";".join(top['reasons'])
    }
    df_row = pd.DataFrame([row])
    if os.path.exists(filename):
        df_row.to_csv(filename, mode='a', header=False, index=False)
    else:
        df_row.to_csv(filename, index=False)
    return filename


In [11]:
def run_interactive_recommender():
    if not WIDGETS_AVAILABLE:
        print("ipywidgets not available — falling back to console input(). Run the fallback cell below.")
        return
    
    # Define widgets
    name_w = widgets.Text(value="", description="Full name:")
    age_w = widgets.IntText(value=30, description="Age:")
    gender_w = widgets.Dropdown(options=["Male","Female","Other"], value="Male", description="Gender:")
    income_w = widgets.IntText(value=300000, description="Annual income (INR):")
    dependents_w = widgets.IntSlider(value=0, min=0, max=10, description="Dependents:")
    goal_w = widgets.Dropdown(options=["Child education","Retirement","Wealth creation","Protection","Legacy","Savings"], value="Retirement", description="Goal:")
    risk_w = widgets.Dropdown(options=["low","medium","high"], value="low", description="Risk tolerance:")
    existing_cover_w = widgets.IntText(value=0, description="Existing cover (INR):")
    horizon_w = widgets.IntSlider(value=10, min=1, max=50, description="Horizon (yrs):")
    
    submit_btn = widgets.Button(description="Get Recommendation", button_style="success")
    output = widgets.Output()
    
    form_items = widgets.VBox([
        name_w, age_w, gender_w, income_w, dependents_w,
        goal_w, risk_w, existing_cover_w, horizon_w, submit_btn, output
    ])
    
    def on_submit(b):
        with output:
            clear_output()
            # simple validation
            if not name_w.value or age_w.value < 18 or age_w.value > 100:
                display(Markdown("**Please enter a valid name and age (18-100).**"))
                return
            user_profile = {
                "name": name_w.value.strip(),
                "age": int(age_w.value),
                "gender": gender_w.value,
                "income": float(income_w.value),
                "dependents": int(dependents_w.value),
                "goal": goal_w.value,
                "risk_tolerance": risk_w.value,
                "existing_cover": float(existing_cover_w.value),
                "time_horizon_years": int(horizon_w.value)
            }
            # run recommender
            recs = recommend_policies(user_profile, policy_catalog, top_k=3)
            top = recs.iloc[0].to_dict()
            explanation = format_explanation(user_profile, top)
            # display results
            display(Markdown(f"### ✅ Top recommendation: **{top['policy_name']}** ({top['policy_type']}) — Confidence: {round(top['confidence_pct'],1)}%"))
            display(Markdown(f"**Explanation:**\n\n{explanation}"))
            display(Markdown("**Other suggestions:**"))
            for i in range(1, len(recs)):
                r = recs.iloc[i]
                display(Markdown(f"- **{r['policy_name']}** ({r['policy_type']}) — Confidence: {round(r['confidence_pct'],1)}% — {r['description']}"))
            # save
            fn = save_recommendation(user_profile, recs)
            display(Markdown(f"Saved recommendation to `{fn}`"))
    
    submit_btn.on_click(on_submit)
    display(form_items)

# To launch the interactive form, run:
# run_interactive_recommender()


In [22]:
# Small example catalog — extend with real LIC details later
policy_catalog = pd.DataFrame([
    {"policy_type":"Endowment", "policy_name":"Jeevan Anand (Endowment)", "min_age":18, "max_age":65,
     "goal_fit":["retirement","education","savings"], "risk":"low", "liquidity":"low",
     "description":"Savings + life cover. Good for long-term goals with guaranteed maturity benefit."},

    {"policy_type":"Term", "policy_name":"LIC Term Plan (Pure Term)", "min_age":18, "max_age":65,
     "goal_fit":["protection","family safety"], "risk":"low", "liquidity":"none",
     "description":"Pure protection with high cover for low premium. Best for protection of dependents."},

    {"policy_type":"ULIP", "policy_name":"LIC ULIP (Unit Linked)", "min_age":18, "max_age":60,
     "goal_fit":["wealth creation","investment"], "risk":"medium-high", "liquidity":"medium",
     "description":"Market-linked returns + life cover. Good for long-term wealth goals with market risk."},

    {"policy_type":"Whole Life", "policy_name":"LIC Whole Life", "min_age":18, "max_age":75,
     "goal_fit":["legacy","retirement"], "risk":"low", "liquidity":"low",
     "description":"Lifelong cover with benefits on death; useful for legacy planning."}
])
policy_catalog




,policy_type,policy_name,min_age,max_age,goal_fit,risk,liquidity,description
0,Endowment,Jeevan Anand (Endowment),18,65,"[retirement, education, savings]",low,low,Savings + life cover. Good for long-term goals...
1,Term,LIC Term Plan (Pure Term),18,65,"[protection, family safety]",low,none,Pure protection with high cover for low premiu...
2,ULIP,LIC ULIP (Unit Linked),18,60,"[wealth creation, investment]",medium-high,medium,Market-linked returns + life cover. Good for l...
3,Whole Life,LIC Whole Life,18,75,"[legacy, retirement]",low,low,Lifelong cover with benefits on death; useful ...


In [24]:
# run this cell to get input prompts in the notebook output area
run_console_recommender()


Full name:  samruddhi santosh jadhav
Age:  21
Gender (M/F/Other):  f
Annual income (INR):  1200000
Number of dependents:  1


Primary goal options: Child education, Retirement, Wealth creation, Protection, Legacy, Savings


Primary goal:  wealth creation
Risk tolerance (low/medium/high):  medium
Existing cover (INR):  0
Time horizon (years):  5



=== Recommendation ===
Top: LIC ULIP (Unit Linked) (ULIP) — Confidence: 100.0%

Explanation:
 Dear samruddhi,

Recommended: LIC ULIP (Unit Linked) (ULIP) — Confidence: 100.0%

Why this fits:
- Market-linked returns + life cover. Good for long-term wealth goals with market risk.
- Key reasons: goal_match, risk_ok, horizon_short

Suggestion: Consider this if your primary goal is 'wealth creation' and your risk preference is 'medium'.

Other suggestions:
- LIC Term Plan (Pure Term) (Term) — Confidence: 85.7% — Pure protection with high cover for low premium. Best for protection of dependents.
- Jeevan Anand (Endowment) (Endowment) — Confidence: 0.0% — Savings + life cover. Good for long-term goals with guaranteed maturity benefit.

Saved to recommendations_log.csv


C:\Users\hp\AppData\Local\Temp\ipykernel_41276\4230399914.py:4: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  "timestamp": datetime.utcnow().isoformat(),
